## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 5
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_5


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 23.8 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Buffalo Bills,5,2022,30.4,71.4,8.1,324.0,104.8,12.3,12.2,...,9.4,77.8,21.4,12.2,12.2,0.109890,2.8,0.8,0.75,3.5
1,Philadelphia Eagles,5,2022,27.0,66.7,8.5,259.8,97.4,12.6,17.6,...,9.6,105.0,24.8,14.6,12.8,0.080808,0.8,1.0,1.20,2.0


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[5]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_5,0.962578,0.382366,0.218974,0.370823,0.412959,0.469540,0.481661
19,San Francisco 49ers,2022,week_5,0.809015,0.191719,0.117419,0.247231,0.368553,0.346788,0.423611
5,Kansas City Chiefs,2022,week_5,0.512650,0.189624,0.150847,0.295400,0.400505,0.309805,0.414523


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_5,0.962578,0.382366,0.218974,0.370823,0.412959,0.469540,0.481661
19,San Francisco 49ers,2022,week_5,0.809015,0.191719,0.117419,0.247231,0.368553,0.346788,0.423611
5,Kansas City Chiefs,2022,week_5,0.512650,0.189624,0.150847,0.295400,0.400505,0.309805,0.414523
20,Tampa Bay Buccaneers,2022,week_5,0.353496,0.260437,0.122472,0.254023,0.360777,0.270241,0.359867
1,Philadelphia Eagles,2022,week_5,0.316507,0.239004,0.158223,0.271283,0.359351,0.268874,0.353996
26,Dallas Cowboys,2022,week_5,0.207159,0.187503,0.100277,0.207616,0.355133,0.211538,0.336636
13,Baltimore Ravens,2022,week_5,0.014602,0.094888,0.071369,0.209501,0.354917,0.149055,0.312377
17,Denver Broncos,2022,week_5,0.052940,0.153300,0.109139,0.201198,0.349232,0.173162,0.312195
4,Los Angeles Chargers,2022,week_5,0.007170,0.117404,0.071707,0.192026,0.350949,0.147851,0.307977
21,Cincinnati Bengals,2022,week_5,0.012436,0.064974,0.061429,0.179672,0.347577,0.133217,0.305684
